<a href="https://colab.research.google.com/github/clancyvoldsoy/Cubadownloader/blob/main/Cubadownloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importar las librerías necesarias
from google.colab import drive
from google.colab import files
import requests
from tqdm import tqdm
import ipywidgets as widgets
from IPython.display import display
import os
import subprocess

# Montar Google Drive
drive.mount('/content/drive')

# Instalar aria2
!apt-get install -y aria2

# Crear widgets para la interfaz
link_input = widgets.Text(
    description='URL:',
    placeholder='Introduce el enlace aquí'
)
magnet_input = widgets.Text(
    description='Magnet:',
    placeholder='Introduce el enlace magnet aquí'
)
download_button = widgets.Button(description='Descargar Archivo')
magnet_button = widgets.Button(description='Descargar Magnet')
output = widgets.Output()

# Función para descargar archivos
def download_file(url, destination):
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024
    t = tqdm(total=total_size, unit='iB', unit_scale=True)
    with open(destination, 'wb') as file:
        for data in response.iter_content(block_size):
            t.update(len(data))
            file.write(data)
    t.close()
    if total_size != 0 and t.n != total_size:
        print("ERROR, algo salió mal")
    else:
        print("Descarga completada")

# Función para manejar la descarga de archivos
def on_download_button_clicked(b):
    with output:
        output.clear_output()
        url = link_input.value
        if url:
            destination = '/content/drive/MyDrive/' + os.path.basename(url)
            download_file(url, destination)

# Función para manejar la descarga de enlaces magnet
def on_magnet_button_clicked(b):
    with output:
        output.clear_output()
        magnet = magnet_input.value
        if magnet:
            destination = '/content/drive/MyDrive/'
            process = subprocess.Popen(['aria2c', '--disable-ipv6', magnet, '-d', destination], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            for line in iter(process.stdout.readline, ''):
                print(line, end='')
            process.stdout.close()
            process.wait()
            print("Descarga de magnet completada")

# Asignar funciones a los botones
download_button.on_click(on_download_button_clicked)
magnet_button.on_click(on_magnet_button_clicked)

# Organizar la interfaz en un layout
ui = widgets.VBox([
    widgets.HBox([link_input, download_button]),
    widgets.HBox([magnet_input, magnet_button]),
    output
])

# Mostrar la interfaz
display(ui)
